In [1]:
import Hartmannb

The result of custom_function at [0.95, 0.95, 0.95, 0.95, 0.95, 0.95] is 0.00011397265188861638


In [2]:
from botorch.test_functions.base import BaseTestProblem
from botorch.test_functions import Hartmann
import numpy as np 
import torch 
import random 
from botorch.models import SingleTaskGP
from botorch.models.transforms.input import Normalize
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
# from botorch.fit import fit_gpytorch_model 
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

In [3]:
from botorch.acquisition.objective import ScalarizedPosteriorTransform

In [4]:
from scipy import stats
import matplotlib.pyplot as plt
from functools import partial
import torch.nn as nn
from botorch.models import MultiTaskGP
import scipy.stats
from botorch.acquisition import AcquisitionFunction
from botorch.acquisition import LogExpectedImprovement
import pandas as pd

In [5]:
import os

In [6]:
from botorch.acquisition import qLogNoisyExpectedImprovement

In [7]:
from botorch.acquisition import qLogExpectedImprovement


In [8]:
class NoisyShiftedAckley(BaseTestProblem):
    dim = 5   # 维度设置为10
    _bounds = [(-32.768, 32.768)] * dim  # Ackley 函数的输入范围

    def __init__(self, noise_std=0.1, shift=None, negate=False):
        super().__init__(negate=negate)
        self.noise_std = noise_std
        self.negate = negate  # 是否取负
        # 如果没有提供shift，就使用0向量
        if shift is not None:
            self.shift = torch.tensor(shift, dtype=torch.float32)
        else:
            self.shift = torch.zeros(self.dim, dtype=torch.float32)

    def evaluate_true(self, X):
        """评估真实的Ackley函数值"""
        # 确保shift和X在同一个设备上
        self.shift = self.shift.to(X.device)

        a = 20
        b = 0.2
        c = 2 * torch.pi
        n = X.shape[-1]

        # 打印维度信息
        print("X shape:", X.shape)
        print("Shift shape:", self.shift.shape)

        sum1 = torch.sum((X + self.shift) ** 2, dim=-1)
        sum2 = torch.sum(torch.cos(c * (X + self.shift)), dim=-1)
        term1 = -a * torch.exp(-b * torch.sqrt(sum1 / n))
        term2 = -torch.exp(sum2 / n)
        ackley_value = term1 + term2 + a + torch.exp(torch.tensor(1.0))
        return ackley_value

    def add_noise(self, Y):
        """在评估的Ackley函数值上添加噪声"""
        noise = torch.randn_like(Y) * self.noise_std
        return Y + noise

    def forward(self, X):
        """计算带噪声和shift的Ackley函数值，并根据需要取负"""
        true_values = self.evaluate_true(X)
        noisy_values = self.add_noise(true_values)

        # 如果需要取负，则返回负值
        if self.negate:
            return -noisy_values
        return noisy_values

In [9]:
ackley_fn1 = NoisyShiftedAckley(noise_std=0, shift=torch.tensor([0, 0, 0, 0, 0]), negate=True)
ackley_fn2 = NoisyShiftedAckley(noise_std=0, shift=torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0]), negate=True)
ackley_fn3 = NoisyShiftedAckley(noise_std=0.2, shift=torch.tensor([0, 0, 0, 0, 0]), negate=True)

C:\Users\10925\AppData\Local\Temp\ipykernel_21052\2339952838.py:11: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [10]:
initial_points_task11 = [Hartmannb.generate_initial_samples_with_custom_function2(5,5, 5, -32.768, 32.768, seed=i)[0] for i in range(10)]
initial_points_task22 = None  # 如果你没有任务2的初始点，可以设置为 None
initial_points_task23 = [Hartmannb.generate_initial_samples_with_custom_function2(20,20, 5, -32.768, 32.768, seed=i)[0] for i in range(20)]

In [11]:
def fit_multi_task_gp_model_and_select_next_point(train_x, train_obj, device, task):
    input_dim = train_x.shape[1]
    outcome_dim = train_obj.shape[1]
    
    num_samples = train_x.size(0)
    task = task.expand(num_samples)  # 扩展 task 张量以匹配 train_x 的样本数

    # 将任务维度拼接到 train_x
    train_x_with_task = torch.cat([train_x, task.unsqueeze(-1)], dim=-1)

    # 修改 Normalize 变换的维度，以匹配拼接后的 train_x 维度
    input_transform = Normalize(d=train_x_with_task.shape[1])

    # 创建多任务模型，区分不同任务的训练数据
    model = MultiTaskGP(
        train_X=train_x_with_task,
        train_Y=train_obj,
        task_feature=train_x_with_task.shape[1] - 1,  # 任务特征索引
        input_transform=input_transform,  # 使用修改后的输入变换
        outcome_transform=Standardize(m=outcome_dim)
    ).to(device)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()

    X_baseline = train_x[train_obj.flatten() == best_value.item()]

    qLogNEI = qLogNoisyExpectedImprovement(model=model, best_f=best_value, X_baseline=X_baseline)
    new_point_analytic, _ = optimize_acqf(
        acq_function=qLogNEI,
        bounds=torch.tensor([[-32.768] * input_dim, [32.768] * input_dim], device=device, dtype=torch.float64),
        q=1,
        num_restarts=20,
        raw_samples=100,
        options={},
    )

    # 确保 new_point_analytic 是二维张量，形状为 (1, input_dim)
    new_point_analytic = new_point_analytic.unsqueeze(0)  # 确保形状为 (1, input_dim)

    return new_point_analytic


In [12]:
def fit_multi_task_gp_model_and_select_next_point1(train_x, train_obj, device, task):
    input_dim = train_x.shape[1]
    outcome_dim = train_obj.shape[1]
    
    num_samples = train_x.size(0)
    task = task.expand(num_samples)  # 扩展 task 张量以匹配 train_x 的样本数

    # 将任务维度拼接到 train_x
    train_x_with_task = torch.cat([train_x, task.unsqueeze(-1)], dim=-1)

    # 修改 Normalize 变换的维度，以匹配拼接后的 train_x 维度
    input_transform = Normalize(d=train_x_with_task.shape[1])

    # 创建和拟合多任务高斯过程模型
    model = MultiTaskGP(
        train_X=train_x_with_task,  # 使用拼接后的 train_x_with_task
        train_Y=train_obj,
        task_feature=-1,  # 假设最后一个维度是任务特征
        input_transform=input_transform,  # 使用匹配维度的 Normalize
        outcome_transform=Standardize(m=1),  # 假设输出为1维
    ).to(device)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()

    # 使用 ScalarizedPosteriorTransform 来标量化多输出模型
    weights = torch.ones(1, device=device)  # 假设输出为1维
    posterior_transform = ScalarizedPosteriorTransform(weights=weights)

    # 使用 qLogExpectedImprovement 选择下一个采样点
    qLogEI = qLogExpectedImprovement(model=model, best_f=best_value, posterior_transform=posterior_transform)

    # 修改上下界为 Ackley 函数的上下界 [-32.768, 32.768]
    lower_bounds = [-32.768] * input_dim  # 根据输入的维度设置下界
    upper_bounds = [32.768] * input_dim   # 根据输入的维度设置上界
    
    bounds = torch.tensor([lower_bounds, upper_bounds], device=device, dtype=torch.float64)

    new_point_analytic, _ = optimize_acqf(
        acq_function=qLogEI,
        bounds=bounds,
        q=1,  # 采样一个点
        num_restarts=20,
        raw_samples=100,
        options={},
    )
    
    return new_point_analytic.squeeze(0)

    


In [13]:
results0 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task23,  # 传递生成的初始样本
    fit_task_fn=fit_multi_task_gp_model_and_select_next_point,
    device='cuda',
    task_type='multi'
)

X shape: torch.Size([5, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([20, 5])
Shift shape: torch.Size([5])
tensor([[ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904],
        [  5.6108,  -6.4440,  28.5213,  20.4439, -32.7603],
        [ 28.2527,  19.6542, -20.1663, -21.0405,  -8.5432],
        [-32.2330, -16.9624,   8.3908,  12.8768,  10.8531],
        [-21.3587,  25.0746,  17.1102,   3.0757,   6.3487],
        [ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904],
        [  5.6108,  -6.4440,  28.5213,  20.4439, -32.7603],
        [ 28.2527,  19.6542, -20.1663, -21.0405,  -8.5432],
        [-32.2330, -16.9624,   8.3908,  12.8768,  10.8531],
        [-21.3587,  25.0746,  17.1102,   3.0757,   6.3487],
        [ 16.6030, -28.5348, -11.4787, -27.7750,  -5.1106],
        [ 15.2134,  15.0031,   3.4497,  29.0767, -19.7940],
        [-10.4568, -13.3433, -25.4652,  -4.5841,  18.0279],
        [-15.2011,  20.7962,   6.1643, -30.5944, -27.6070],
        [ 10.4527, -24.5044, -30.2289,   6.14

C:\Users\10925\AppData\Roaming\Python\Python311\site-packages\linear_operator\utils\interpolation.py:71: UserWarning:

The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:80.)

C:\Users\10925\AppData\Roaming\Python\Python311\site-packages\linear_operator\utils\interpolation.py:71: UserWarning:

torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:623.)

F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch

X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
Nr. of optimization run: 0
New candidates are: tensor([[ -3.7814, -23.5248,   4.7259,  -2.6196,  -0.3774]], device='cuda:0',
       dtype=torch.float64)
Iteration 0 took 4.03 seconds


F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.multitask.MultiTaskGP'>. Setting `cache_root = False`.



X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
Nr. of optimization run: 1
New candidates are: tensor([[ -4.3037, -17.9005,   4.8802,  -1.9246,   8.9685]], device='cuda:0',
       dtype=torch.float64)
Iteration 1 took 1.25 seconds


...

X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
Nr. of optimization run: 22
New candidates are: tensor([[-2.0418, -2.8191, -0.1000, -0.7403,  2.1589]], device='cuda:0',
       dtype=torch.float64)
Iteration 22 took 7.42 seconds


F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.multitask.MultiTaskGP'>. Setting `cache_root = False`.



X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
Nr. of optimization run: 23
New candidates are: tensor([[-2.3223, -4.3399,  1.5890, -0.1383,  2.1278]], device='cuda:0',
       dtype=torch.float64)
Iteration 23 took 6.64 seconds


F:\anaconda\Lib\site-packages\botorch\acquisition\cached_cholesky.py:89: RuntimeWarning:

`cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.multitask.MultiTaskGP'>. Setting `cache_root = False`.



X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
Nr. of optimization run: 24
New candidates are: tensor([[-2.4844, -1.0537, -1.1970, -0.3753,  2.0274]], device='cuda:0',
       dtype=torch.float64)
Iteration 24 took 5.36 seconds
最优点: [-21.296891618859263, -20.94337719240681, -19.343187515149438, -19.343187515149438, -18.24354018584188, -17.335636081542457, -15.193036719241633, -14.047900168921664, -14.047900168921664, -14.047900168921664, -12.559924085690012, -11.684076133070896, -10.493436246002064, -10.493436246002064, -10.493436246002064, -10.18987823194417, -10.18987823194417, -10.18987823194417, -10.18987823194417, -10.18987823194417, -10.173128182864591, -10.133013358995175, -9.720095973496628, -9.372345843177035, -9.372345843177035, -8.290931765044487, -8.213057958217695, -7.22893143747927, -7.22893143747927, -7.108732809360598]
Bayes Best Values Over All Experiments:
[[-20.516918177000896, -20.516918177000896, -20

In [14]:
def fit_gp_model_and_select_next_point(train_x, train_obj, device):
    # 创建和拟合高斯过程模型
    model = SingleTaskGP(
        train_X=train_x,
        train_Y=train_obj,
        input_transform=Normalize(d=train_x.shape[1]),  # 使用输入的维度
        outcome_transform=Standardize(m=1),  # 输出为一维
    ).to(device)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()


    X_baseline = train_x[train_obj.flatten() == best_value.item()]

    qLogNEI = qLogNoisyExpectedImprovement(model=model, best_f=best_value, X_baseline=X_baseline)
    # 修改上下界为 Ackley 函数的上下界 [-32.768, 32.768]
    lower_bounds = [-32.768] * train_x.shape[1]  # 根据输入的维度设置下界
    upper_bounds = [32.768] * train_x.shape[1]   # 根据输入的维度设置上界
    
    bounds = torch.tensor([lower_bounds, upper_bounds], device=device, dtype=torch.float64)
    
    new_point_analytic, _ = optimize_acqf(
        acq_function=qLogNEI,
        bounds=bounds,
        q=1,  # 采样一个点
        num_restarts=20,
        raw_samples=100,
        options={},
    )
    
    return new_point_analytic.squeeze(0)


In [15]:
def fit_gp_model_and_select_next_point1(train_x, train_obj, device):
    # 创建和拟合高斯过程模型
    model = SingleTaskGP(
        train_X=train_x,
        train_Y=train_obj,
        input_transform=Normalize(d=train_x.shape[1]),  # 使用输入的维度
        outcome_transform=Standardize(m=1),  # 输出为一维
    ).to(device)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()



    qLogEI = qLogExpectedImprovement(model=model, best_f=best_value)

    # 修改上下界为 Ackley 函数的上下界 [-32.768, 32.768]
    lower_bounds = [-32.768] * train_x.shape[1]  # 根据输入的维度设置下界
    upper_bounds = [32.768] * train_x.shape[1]   # 根据输入的维度设置上界
    
    bounds = torch.tensor([lower_bounds, upper_bounds], device=device, dtype=torch.float64)
    
    new_point_analytic, _ = optimize_acqf(
        acq_function=qLogEI,
        bounds=bounds,
        q=1,  # 采样一个点
        num_restarts=20,
        raw_samples=100,
        options={},
    )
    
    return new_point_analytic.squeeze(0)

In [16]:
results1 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task22,  # 传递生成的初始样本
    fit_task_fn=fit_gp_model_and_select_next_point,
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)


X shape: torch.Size([5, 5])
Shift shape: torch.Size([5])
tensor([[ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904],
        [  5.6108,  -6.4440,  28.5213,  20.4439, -32.7603],
        [ 28.2527,  19.6542, -20.1663, -21.0405,  -8.5432],
        [-32.2330, -16.9624,   8.3908,  12.8768,  10.8531],
        [-21.3587,  25.0746,  17.1102,   3.0757,   6.3487]], device='cuda:0',
       dtype=torch.float64)
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
Nr. of optimization run: 0
New candidates are: tensor([[-15.2143,   1.3246,   4.3613,  -2.6716,  22.8701]], device='cuda:0',
       dtype=tor

In [17]:
def fit_gp_model_and_select_next_point2(train_x, train_obj, device):
        # 生成样本点
    initial_samples_np = Hartmannb.generate_initial_samples(num_samples=1, dim=5, lower_bound=-32.768, upper_bound=32.768)
    
    # 转换为张量
    initial_samples_tensor = torch.tensor(initial_samples_np, dtype=torch.float64)
    return initial_samples_tensor.squeeze(0)

In [18]:
results2 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task22,  # 传递生成的初始样本
    fit_task_fn=fit_gp_model_and_select_next_point2,
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)

X shape: torch.Size([5, 5])
Shift shape: torch.Size([5])
tensor([[ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904],
        [  5.6108,  -6.4440,  28.5213,  20.4439, -32.7603],
        [ 28.2527,  19.6542, -20.1663, -21.0405,  -8.5432],
        [-32.2330, -16.9624,   8.3908,  12.8768,  10.8531],
        [-21.3587,  25.0746,  17.1102,   3.0757,   6.3487]], device='cuda:0',
       dtype=torch.float64)
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
Nr. of optimization run: 0
New candidates are: tensor([[ 14.8769, -31.4564,  -6.1706, -13.3107,  32.0271]], device='cuda:0',
       dtype=tor

C:\Users\10925\AppData\Local\Temp\ipykernel_21052\3067467820.py:6: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



New candidates are: tensor([[ -9.2406,  -8.5327,  30.9721,  22.1653, -28.3397]], device='cuda:0',
       dtype=torch.float64)
Iteration 18 took 0.00 seconds
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
Nr. of optimization run: 19
New candidates are: tensor([[-15.4046,   4.9203, -25.9994,  -9.2031, -13.4887]], device='cuda:0',
       dtype=torch.float64)
Iteration 19 took 0.00 seconds
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
Nr. of optimization run: 20
New candidates are: tensor([[ -8.3259,  28.4486, -20.0782,  13.0785,  -5.1870]], device='cuda:0',
       dtype=torch.float64)
Iteration 20 took 0.01 seconds
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
Nr. of optimization run: 21
New candidates are: tensor([[  2.8003,   6.9482,  15.9459,  27.7870, -22.0380]], device='cuda:0',
     

In [19]:
from pyDOE2 import lhs
from scipy.stats import qmc

In [20]:
# 生成 LHS 样本并缩放到指定上下界
def generate_lhs_samples(num_features, num_samples, lower_bounds, upper_bounds):
    # 生成 LHS 样本
    lhs_sampler = qmc.LatinHypercube(d=num_features)
    samples = lhs_sampler.random(n=num_samples)
    
    # 缩放样本到指定的上下界
    scaled_samples = qmc.scale(samples, lower_bounds, upper_bounds)
    
    return scaled_samples


# 管理 LHS 样本并按顺序检索它们的类
class SampleProvider:
    def __init__(self, num_features, num_samples):
        self.samples_np = generate_lhs_samples(num_features, num_samples, lower_bounds, upper_bounds)
        self.samples_tensor = torch.tensor(self.samples_np, dtype=torch.float64)
        self.current_index = 0

    def get_next_sample(self):
        if self.current_index < self.samples_tensor.size(0):
            sample = self.samples_tensor[self.current_index]
            self.current_index += 1
            return sample
        else:
            raise IndexError("没有更多样本可用了。")


In [21]:
initial_points_task1 = [Hartmannb.generate_initial_samples_with_custom_function(5, 20, 5, -32.768, 32.768, seed=i)[0] for i in range(10)]
initial_points_task2 = None  # 如果你没有任务2的初始点，可以设置为 None

In [22]:
# 示例使用
num_features = 5
num_samples = 1250
lower_bounds = [-32.768] * 5
upper_bounds = [32.768] * 5
sample_provider = SampleProvider(num_features, num_samples)



# 检索一个样本（每次调用函数时都会这样做）
next_sample = fit_gp_model_and_select_next_point2(None, None, None)
print(next_sample)

tensor([-23.8515,  -6.6344, -10.7426,  -0.4613, -25.3295], device='cuda:0',
       dtype=torch.float64)


C:\Users\10925\AppData\Local\Temp\ipykernel_21052\3067467820.py:6: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [23]:
# 类似你提供的函数
def zj(train_x, train_obj, device):
    return sample_provider.get_next_sample()

In [24]:
results3 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    fit_task_fn=zj,  # 你的模型拟合函数
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task1,  # 传递生成的初始样本
    initial_points_task2=initial_points_task2,  # 传递生成的初始样本
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)





X shape: torch.Size([5, 5])
Shift shape: torch.Size([5])
tensor([[  6.8479,  -7.7774, -30.0446, -18.5324, -21.5619],
        [ 24.9812,  25.9686,  26.6705,   7.7149,  -5.3534],
        [-23.5003,  11.6991,   6.1859,  -5.2805,  31.8218],
        [-11.5061, -29.8020, -15.2383,  21.7279,   7.9949],
        [ -2.3895,   4.0977,  13.3367, -23.2132,  -6.6525]], device='cuda:0')
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
Nr. of optimization run: 0
New candidates are: tensor([[  4.5887, -18.1201,  28.1352, -32.1095, -10.3999]], device='cuda:0',
       dtype=torch.float64)
Iteration 0 took

In [25]:
results4 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task23,  # 传递生成的初始样本
    fit_task_fn=fit_multi_task_gp_model_and_select_next_point1,
    device='cuda',
    task_type='multi'
)

X shape: torch.Size([5, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([20, 5])
Shift shape: torch.Size([5])
tensor([[ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904],
        [  5.6108,  -6.4440,  28.5213,  20.4439, -32.7603],
        [ 28.2527,  19.6542, -20.1663, -21.0405,  -8.5432],
        [-32.2330, -16.9624,   8.3908,  12.8768,  10.8531],
        [-21.3587,  25.0746,  17.1102,   3.0757,   6.3487],
        [ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904],
        [  5.6108,  -6.4440,  28.5213,  20.4439, -32.7603],
        [ 28.2527,  19.6542, -20.1663, -21.0405,  -8.5432],
        [-32.2330, -16.9624,   8.3908,  12.8768,  10.8531],
        [-21.3587,  25.0746,  17.1102,   3.0757,   6.3487],
        [ 16.6030, -28.5348, -11.4787, -27.7750,  -5.1106],
        [ 15.2134,  15.0031,   3.4497,  29.0767, -19.7940],
        [-10.4568, -13.3433, -25.4652,  -4.5841,  18.0279],
        [-15.2011,  20.7962,   6.1643, -30.5944, -27.6070],
        [ 10.4527, -24.5044, -30.2289,   6.14

In [26]:
results5 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=10,
    n=25,
    obj_fn1=ackley_fn3,
    obj_fn2=ackley_fn2,
    obj_fn3=ackley_fn1,
    initial_points_task1=initial_points_task11,  # 传递生成的初始样本
    initial_points_task2=initial_points_task22,  # 传递生成的初始样本
    fit_task_fn=fit_gp_model_and_select_next_point1,
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)


X shape: torch.Size([5, 5])
Shift shape: torch.Size([5])
tensor([[ -1.6314,   6.0637,  -0.3618, -12.1994,  29.8904],
        [  5.6108,  -6.4440,  28.5213,  20.4439, -32.7603],
        [ 28.2527,  19.6542, -20.1663, -21.0405,  -8.5432],
        [-32.2330, -16.9624,   8.3908,  12.8768,  10.8531],
        [-21.3587,  25.0746,  17.1102,   3.0757,   6.3487]], device='cuda:0',
       dtype=torch.float64)
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
X shape: torch.Size([1, 5])
Shift shape: torch.Size([5])
Nr. of optimization run: 0
New candidates are: tensor([[-22.2080,  15.0977,  12.4882,   1.8420,  13.7894]], device='cuda:0',
       dtype=tor

In [27]:
results_list = [results0,results1, results2, results3, results4, results5]
results_names = ['results0','results1', 'results2', 'results3' ,'results4', 'results5']

# 初始化一个空字典来存储展平后的数据
flattened_results = {}

# 遍历所有结果数组
for i, results in enumerate(results_list):
    if isinstance(results, torch.Tensor):
        results_np = results.cpu().numpy()  # 转为 numpy 数组
    else:
        results_np = results  # 如果已经是 numpy 数组，直接使用
    
    # 获取数组的维度
    shape_len = len(results_np.shape)
    
    if shape_len == 3:
        # 3D 数组的形状 (num_experiments, num_samples, num_metrics)
        num_experiments, num_samples, num_metrics = results_np.shape
        # 展平为 2D 数组
        flattened_array = results_np.reshape(num_experiments * num_samples, num_metrics)
    elif shape_len == 2:
        # 2D 数组的形状 (num_experiments, num_metrics)
        flattened_array = results_np
        num_experiments, num_metrics = flattened_array.shape
    else:
        raise ValueError(f"Unexpected number of dimensions: {shape_len}")
    
    # 将每个数组的列命名为 'results1_Metric_1', 'results1_Metric_2', ...
    columns = [f"{results_names[i]}_Metric_{j+1}" for j in range(num_metrics)]
    
    # 将展平后的数组存入字典，方便后面转换为 DataFrame
    flattened_results.update({col: flattened_array[:, j] for j, col in enumerate(columns)})

# 创建 DataFrame
df_results = pd.DataFrame(flattened_results)

# 文件名
csv_filename = 'all_results_ackley5.噪声2.csv'

# 检查 CSV 文件是否已经存在
if os.path.exists(csv_filename):
    # 如果文件存在，先读取现有文件
    df_existing = pd.read_csv(csv_filename)
    # 合并新结果到现有的 DataFrame 中
    df_results = pd.concat([df_existing, df_results], ignore_index=True)

# 保存或追加为 CSV 文件
df_results.to_csv(csv_filename, index=False)

print(f"所有结果已保存到 {csv_filename} 文件中。")

所有结果已保存到 all_results_ackley5.噪声2.csv 文件中。
